In [42]:
import warnings
warnings.filterwarnings('ignore')

import io
import random
import numpy as np
import mxnet as mx
import gluonnlp as nlp
from bert import data, model
import time
from mxnet.gluon.utils import split_and_load 
import pandas as pd
import numpy as np
import swifter

val = pd.read_hdf('cleaned.h5', 'val')
train = pd.read_hdf('cleaned.h5', 'train')
candid = pd.read_hdf('cleaned.h5', 'candid')
candid = candid.reset_index(drop=True)

train = train.replace('\\n','', regex=True)
train = train.replace('\\r','', regex=True)
candid = candid.replace('\\n','', regex=True)
candid = candid.replace('\\r','', regex=True)
val = val.replace('\\n','', regex=True)
val = val.replace('\\r','', regex=True)

candid.abstract = candid.abstract.swifter.allow_dask_on_strings().apply(lambda x: ''.join([" " if ord(i) < 32 or ord(i) > 126 else i for i in x]))
candid.title = candid.title.swifter.allow_dask_on_strings().apply(lambda x: ''.join([" " if ord(i) < 32 or ord(i) > 126 else i for i in x]))

In [3]:
# bm25_rank_val

In [52]:
bm25_rank_val = np.load('test_recall.npy')

In [5]:
# bm25_rank_train = np.load('train_recall.npy')

np.random.seed(100)
random.seed(100)
mx.random.seed(10000)
# change `ctx` to `mx.cpu()` if no GPU is available.
# ctx = [mx.gpu(0), mx.gpu(1), mx.gpu(2), mx.gpu(3)]
ctx = [mx.gpu(0), mx.gpu(1), mx.gpu(2), mx.gpu(3), mx.gpu(4), mx.gpu(5), mx.gpu(6), mx.gpu(7)]
# ctx = mx.cpu()
bert_base, vocabulary = nlp.model.get_model('bert_12_768_12',#book_corpus_wiki_en_uncased
                                             dataset_name='biobert_v1.1_pubmed_cased', #biobert_v1.1_pubmed_cased
                                             pretrained=True, ctx=ctx, use_pooler=True,
                                             use_decoder=False, use_classifier=False)
bert_classifier = model.classification.BERTClassifier(bert_base, num_classes=2, dropout=0.1)
# only need to initialize the classifier layer.
bert_classifier.classifier.initialize(init=mx.init.Normal(0.02), ctx=ctx)
bert_classifier.hybridize(static_alloc=True)

# softmax cross entropy loss for classification
loss_function = mx.gluon.loss.SoftmaxCELoss()
loss_function.hybridize(static_alloc=True)

metric = mx.metric.Accuracy()
# bert_classifier.load_parameters('bisai/epoch121599.params')
bert_classifier.load_parameters('bisai/512new/epoch121599.params')

In [6]:
# bm25_rank_val[0]

In [62]:
def gen_test_set(start=0,end=20,mask=None):
    idx2pid = candid['paper_id'].to_dict()
    pid2idx = {v: k for k, v in idx2pid.items()}
    test_set = []
    
    des = val.description_text.to_list()
    tit = candid.title.to_list()
    abst = candid.abstract.to_list()
    
    for idx,row in enumerate(bm25_rank_val):
        row = row[start:end]
            
        if mask is not None:
            if mask[idx]:
                continue
                
        for pid in row:
            t = {
                'relavence': 'UNKOWN',
                'qidx': idx,
                'pidx': pid,
                'q_text': des[idx],
                'd_text': tit[pid] + ' ' + abst[pid]
            }
            
            test_set.append(t)
            
        if (idx % 10000 == 0):
            print(idx)
    print(len(bm25_rank_val))
    pd.DataFrame(test_set).to_csv('val_bert_50.tsv', sep='\t', index=False)

In [15]:
def get_score():
    num_discard_samples = 1
    field_separator = nlp.data.Splitter('\t')
    field_indices = [3, 4, 0]
    data_train_raw = nlp.data.TSVDataset(filename='val_bert_50.tsv', 
                                         # change val_bert or test_bert_train_val val_bert_50
                                     field_separator=field_separator,
                                     num_discard_samples=num_discard_samples,
                                     field_indices=field_indices)
    bert_tokenizer = nlp.data.BERTTokenizer(vocabulary, lower=False)
    max_len = 512
    all_labels = ["0", "UNKOWN"]

    # whether to transform the data as sentence pairs.
    # for single sentence classification, set pair=False
    # for regression task, set class_labels=None
    # for inference without label available, set has_label=False
    pair = True
    transform = data.transform.BERTDatasetTransform(bert_tokenizer, max_len,
                                                    class_labels=all_labels,
                                                    has_label=True,
                                                    pad=True,
                                                    pair=pair)


    data_train = data_train_raw.transform(transform)
    # Training the model with only three epochs

    batch_size = 500
    bert_dataloader = mx.gluon.data.DataLoader(data_train, batch_size=batch_size, num_workers=40,
                                               pin_memory=True, prefetch=20000)

    log_interval = 160
    test_log_interval = 320

    start_log_interval_time = time.time()
    new_recall_score = []
    
    cntdy0 = 0

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(bert_dataloader):
        token_ids = split_and_load(token_ids, ctx, even_split=False)
        valid_length = split_and_load(valid_length.astype('float32'), ctx, even_split=False)
        segment_ids = split_and_load(segment_ids, ctx, even_split=False)
        label = [bert_classifier(a, b, c)
                      for a, b, c in zip(token_ids, segment_ids, valid_length)]

        label = [l[:,1].asnumpy() for l in label]
        new_recall_score.extend(label)
        
        for i in range(8):
            cntdy0 += sum(label[i]>0)
            
        if batch_id % 50 == 0:
            print(cntdy0)
            
        if (batch_id + 1) % test_log_interval == 0:
            print('[Batch {}/{}] elapsed {:.2f} s'.format(
                batch_id + 1, len(bert_dataloader),
                time.time() - start_log_interval_time))
            start_log_interval_time = time.time()
    return new_recall_score

In [9]:
def scatter_rows(mask,scores,k=10):
    init_score = np.ones([len(mask),k]) * -1000
    cnt = 0
    for idx,row in enumerate(mask):
        if row==False:
            init_score[idx,:]=scores[cnt,:]
            cnt+=1
    return init_score
# scatter_rows(np.array([True,False,True,False]),
#             np.array([[1,2,3,4,5,6,7,8,9,0],[1,2,3,4,5,6,7,8,9,0]]))

In [16]:
k = 50
# cal first score
gen_test_set(start=0,end=k)
np_recall_score = np.hstack(get_score()).reshape([-1, k])

0
59
129
156
227
301
360
[Batch 320/3443] elapsed 1013.20 s
389
450
488
553
603
666
[Batch 640/3443] elapsed 995.01 s
732
806
844
891
964
1024
1067
[Batch 960/3443] elapsed 992.84 s
1123
1181
1221
1279
1338
1421
[Batch 1280/3443] elapsed 991.76 s
1473
1539
1605
1653
1708
1764
[Batch 1600/3443] elapsed 995.58 s
1828
1884
1926
1992
2039
2091
2136
[Batch 1920/3443] elapsed 994.78 s
2184
2235
2297
2360
2416
2475
[Batch 2240/3443] elapsed 993.73 s
2530
2573
2623
2674
2742
2806
2870
[Batch 2560/3443] elapsed 995.79 s
2922
3003
3086
3130
3196
3239
[Batch 2880/3443] elapsed 996.33 s
3317
3364
3471
3550
3608
3677
[Batch 3200/3443] elapsed 992.99 s
3747
3811
3854
3934
3999


Process ForkPoolWorker-359:
Process ForkPoolWorker-356:
Process ForkPoolWorker-340:
Process ForkPoolWorker-342:
Process ForkPoolWorker-321:
Process ForkPoolWorker-319:
Process ForkPoolWorker-361:
Process ForkPoolWorker-338:
Process ForkPoolWorker-318:
Process ForkPoolWorker-351:
Process ForkPoolWorker-346:
Process ForkPoolWorker-326:
Process ForkPoolWorker-329:
Process ForkPoolWorker-314:
Process ForkPoolWorker-316:
Process ForkPoolWorker-349:
Process ForkPoolWorker-333:
Process ForkPoolWorker-358:
Process ForkPoolWorker-336:
Process ForkPoolWorker-348:
Traceback (most recent call last):
Process ForkPoolWorker-365:


In [49]:
treshold = 10
mask = np.sum(np_recall_score>treshold, axis=1)>0
print(sum(mask), len(mask))

19875 34428


In [ ]:
for i in range(1,1000):
    np.save('mask{}.npy'.format(i), mask)
    np.save('np_recall_score{}.npy'.format(i), np_recall_score)

    np_new_recall_score_idx = np.argsort(-np_recall_score, axis=1)
    bert_rerank_train_val = np.take_along_axis(bm25_rank_val, np_new_recall_score_idx, axis=1)
    val['ans0'] = bert_rerank_train_val[:,0]
    val['ans1'] = bert_rerank_train_val[:,1]
    val['ans2'] = bert_rerank_train_val[:,2]
    def idx2name(idx):
        return candid.iloc[idx].paper_id
    val['ans0'] = val['ans0'].apply(idx2name)
    val['ans1'] = val['ans1'].apply(idx2name)
    val['ans2'] = val['ans2'].apply(idx2name)

    val[['description_id','ans0','ans1','ans2']].to_csv('test_commit_{}_.csv'.format(i),
                                                        index=False, header=None)
    
    start = i * k
    end = (i+1) *k

    print('start=',start,'end=',end)

    gen_test_set(start,end,mask)
    # cal score
    np_recall_score_ext = np.hstack(get_score()).reshape([-1, k])
    np_recall_score_ext = scatter_rows(mask,np_recall_score_ext,k=k)
    
    
    if i>=9:
        treshold = 2
    mask_ = np.sum(np_recall_score_ext>treshold, axis=1)>0
    mask = mask | mask_
    print(sum(mask), len(mask))

    np_recall_score = np.hstack([np_recall_score, np_recall_score_ext])
    print(np_recall_score.shape)
    
    print('\n')


start= 450 end= 500
0
10000
20000
34428
0
10
18
25
33
49
60
[Batch 320/1456] elapsed 992.50 s
68
81
92
101
109
119
[Batch 640/1456] elapsed 990.68 s
130
141
155
165
173
182
193
[Batch 960/1456] elapsed 992.01 s
198
208
220
235
241
258
[Batch 1280/1456] elapsed 994.95 s
272
279
292
303
19875 34428
(34428, 500)


start= 500 end= 550
0
10000
20000
34428
0
10
22
31
36
54
60
[Batch 320/1456] elapsed 997.57 s
71
77
86
94
105
115
[Batch 640/1456] elapsed 991.04 s
131
147
154
166
171
180
196
[Batch 960/1456] elapsed 991.64 s
209
222
229
239
249
256
[Batch 1280/1456] elapsed 993.82 s
263
272
278
286
19875 34428
(34428, 550)


start= 550 end= 600
0
10000
20000
34428
0
10
23
30
36
46
57
[Batch 320/1456] elapsed 995.25 s
68
73
85
96
107
115
[Batch 640/1456] elapsed 993.16 s
121
130
137
146
154
159
167
[Batch 960/1456] elapsed 995.46 s
174
180
189
197
212
223
[Batch 1280/1456] elapsed 993.48 s
232
236
240
244
19875 34428
(34428, 600)


start= 600 end= 650
0
10000
20000
34428
0
6
20
34
40
52
65
[Bat

In [ ]:
1